# 합성곱 신경망 (Convolutional neural network)

## 합성곱 계층 구현하기
- CNN에서 계층사이를 흐르는 데이터는 4차원: (데이터 수, 입력 채널 수, 높이, 너비)
- for문을 이용한다면 합성곱 구현은 복잡해진다
- 책에서는 im2col() 이라는 함수를 이용하여 4차원의 데이터를 2차원으로 바꾸어 계산을 간단히 한다.

#### 책에서 구현한 im2col() 함수
- https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/util.py#L39
- CNN의 순전파에 이용한다

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    # 입력데이터의 형태정보를 가져온다
    N, C, H, W = input_data.shape
    
    # 합성곱 결과의 형태를 계산한다 
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    # np.pad(패딩을 적용할 array, [각 axis에 패딩을 어떻게 적용할지 지정], '옵션')
    # 예) pad()의 리스트 인자의 세번째 튜플 (pad,pad)는 입력데이터 (H,W) 행렬의 위,아래로 행을 'pad'개 만큼 만든다
    # 예) pad()의 리스트 인자의 네번째 튜플 (pad,pad)는 입력데이터 (H,W) 행렬의 왼쪽,오른쪽으로 열을 'pad'개 만큼 만든다
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    
    # im2col()의 결과를 저장할 배열을 만든다
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    # 이 for문에서 하는 일은 각 필터 원소와 곱할 입력 데이터의 원소들을 모아 col에 저장하는 것
    for y in range(filter_h):
        # 필터가 입력데이터 위에서 y 방향으로 움직이는 최대 위치
        y_max = y + stride*out_h
        for x in range(filter_w):
            # 필터가 입력데이터 위에서 x 방향으로 움직이는 최대 위치
            x_max = x + stride*out_w
            
            # filter_xy와 곱합 입력데이터(img[:, :, y:y_max:stride, x:x_max:stride])를 col[:, :, y, x, :, :] 저장
            # y:y_max:stride에서 세번째 stride는 슬라이싱할때 건너뛰는 크기
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride] 

    # col을 후에 똑같은 방법으로 2차원배열로 변환한 필터와 곱하기 위해 형상을 바꾸어준다 
    # col.transpose(0, 4, 5, 1, 2, 3) 결과는 (N, out_h, out_w, C, filter_h, filter_w)의 형태를 갖는 행렬
    # reshape후에는 (N*out_h*out_w, C*filter_h*filter_w)의 2차원 배열이다
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [ ]:
# 아래 코드는 im2col()을 이용하여 (1,3,7,7)의 배열을 (9,75)의 형상을 갖는 배열로 바꾸어준다
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride = 1, pad = 0)
print(col1.shape)

#### 책에서 구현한 col2im() 함수
- https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/util.py#L71
- CNN의 역전파에 이용한다

In [ ]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    
    # 전달된 col은 (N*out_h*out_w, C*filter_h*filter_w)의 2차원 배열
    # reshape() 후에는 (N, out_h, out_w, C, filter_h, filter_w)의 형상을 갖는 배열
    # transpose() 후에는 (N, C, filter_h, filter_w, out_h, out_w)의 형상을 갖는 배열
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

### 합성곱 계층 클래스
- https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/layers.py#L198

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 중간 데이터（backward 시 사용）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 가중치와 편향 매개변수의 기울기
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        # im2col()의 결과로 나온 행렬과 dot()연산하기위해 변형한다
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx